In [11]:
from dotenv import load_dotenv
import os
import bs4
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

load_dotenv()

True

In [6]:
from langchain_openai import ChatOpenAI

In [8]:
llm = ChatOpenAI(model='gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))

In [ ]:


# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()


# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [10]:
response = rag_chain.invoke({"input": "What is Task Decomposition?"})
response["answer"]

'Task decomposition is the process of breaking down a complicated task into smaller, more manageable steps. This can be achieved using techniques like Chain of Thought (CoT), which encourages the model to think step by step, or Tree of Thoughts, which explores multiple reasoning possibilities for each step. By decomposing tasks, it becomes easier to understand and solve complex problems effectively.'

In [18]:
loader = PyPDFLoader("proposals/file_10.pdf")
docs = loader.load()
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()


In [19]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is definition of Paradox of Choice"})
response["answer"]

'The Paradox of Choice is a phenomenon where an abundance of options can lead to less happiness, less satisfaction, and a reduced ability to make a decision. It highlights how having too many choices can overwhelm consumers, making it difficult for them to make a decision at all.'

In [64]:
def extract_pdf(file_path):
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    
    return " ".join([doc.page_content for doc in docs])
    
def summarize(user_input, file_path=None):
    if file_path:
        user_input = extract_pdf(file_path)
    
    print(user_input)
    predefined_sections = {
        "Introduction": "",
        "Objectives": "",
        "Methodology": "",
        "Results": "",
        "Conclusion": ""
    }

    system_prompt = (
        f"You are an assistant for document summarize tasks. Summarize the content and categorize it into these sections: {list(predefined_sections.keys())}"
        "If a section lacks sufficient information, leave it empty and indicate that more input is needed."
        "answer concise."
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )
    chain = prompt | llm
    response = chain.invoke({"input": user_input})

    return response


In [65]:
def read_text_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

# Example usage
file_path = 'summarize.txt'
text_content = read_text_file(file_path)

In [67]:
res = summarize(text_content)

Proposal for Planogram Evaluation Using AI

Introduction
Efficient merchandising is critical for optimizing shelf space, enhancing customer experience, and driving sales. Planograms—visual representations of a store's layout—are integral to achieving this, ensuring product placement aligns with business goals. However, evaluating planograms for compliance, performance, and customer impact is labor-intensive and prone to inconsistencies.

Leveraging artificial intelligence (AI) offers a transformative approach. AI-powered solutions can analyze planograms efficiently, providing actionable insights to optimize layouts, ensure compliance, and boost profitability. This proposal outlines a comprehensive plan for implementing AI-based planogram evaluation.

Objectives
Automate Planogram Evaluation: Reduce the time and effort required to assess planogram compliance and effectiveness.
Improve Accuracy: Minimize errors and inconsistencies inherent in manual evaluations.
Optimize Shelf Performanc

In [69]:
print(res.content)

**Introduction**  
Efficient merchandising is critical for optimizing shelf space, enhancing customer experience, and driving sales. Planograms—visual representations of a store's layout—are integral to achieving this, ensuring product placement aligns with business goals. However, evaluating planograms for compliance, performance, and customer impact is labor-intensive and prone to inconsistencies. Leveraging artificial intelligence (AI) offers a transformative approach to analyze planograms efficiently, providing actionable insights to optimize layouts, ensure compliance, and boost profitability.

**Objectives**  
- Automate Planogram Evaluation: Reduce the time and effort required to assess planogram compliance and effectiveness.  
- Improve Accuracy: Minimize errors and inconsistencies inherent in manual evaluations.  
- Optimize Shelf Performance: Provide actionable insights to enhance product visibility, customer satisfaction, and sales.  
- Enable Continuous Improvement: Use AI 